In [ ]:

import tensorflow as tf
gpu = tf.config.experimental.list_physical_devices('GPU') # 내 컴에 장착된 GPU를 list로 반환
try:
    tf.config.experimental.set_memory_growth(gpu[0], True) # GPU Memory Growth를 Enable
except RuntimeError as e:
    print(e) # Error 발생하면 Error 내용 출력

In [1]:
from summarizer.domain.model.video import Video
import matplotlib

In [2]:
video = Video(url="tests/2.mp4", algorithm="yolov3")
images = video._read_video()
idx, image = next(images)

length : 4625
width : 640
height : 360
fps : 25.0


2022-04-07 20:57:11.649034: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-04-07 20:57:11.649234: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:

features = image.extract()
print(features)

2022-04-07 20:57:13.806317: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


[Feature(current_frame=0, name='backpack', percentage_probability=82.75119662284851, box_points=[296, 147, 449, 288]), Feature(current_frame=0, name='person', percentage_probability=99.84932541847229, box_points=[273, 61, 480, 383]), Feature(current_frame=0, name='backpack', percentage_probability=71.70810103416443, box_points=[278, 106, 474, 376])]


In [ ]:
video = Video(url="tests/2.mp4") 
list_of_feature = video.extract_feature()
video.shorten(video_feature=list_of_feature, must_include_feature=["person"])

In [8]:
import boto3


def create_movie_table(dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")

    table = dynamodb.create_table(
        TableName='Feature',
        KeySchema=[
            {
                'AttributeName': 'key',
                'KeyType': 'HASH'  # Partition key
            },
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'key',
                'AttributeType': 'S'
            },
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 10,
            'WriteCapacityUnits': 10
        }
    )
    return table


if __name__ == '__main__':
    movie_table = create_movie_table()
    print("Table status:", movie_table.table_status)

Table status: ACTIVE


In [4]:
import boto3
from summarizer.infrastructure.repository.feature_repository import FeatureRepository
dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")
movie_table = dynamodb.Table('Feature')
repo = FeatureRepository(movie_table)

In [5]:
repo.put(features)

In [6]:
movie_table.scan()

{'Items': [],
 'Count': 0,
 'ScannedCount': 0,
 'ResponseMetadata': {'RequestId': '438130eb-3058-4bc7-a98d-9c7e2a8438d8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 07 Apr 2022 11:57:37 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'x-amz-crc32': '583912891',
   'x-amzn-requestid': '438130eb-3058-4bc7-a98d-9c7e2a8438d8',
   'content-length': '39',
   'server': 'Jetty(9.4.18.v20190429)'},
  'RetryAttempts': 0}}

In [6]:
x = repo.get('$2022/04/07, 20:46:55')

In [7]:
print(x)

url='tests/2.mp4' parameter={'length': 4625, 'width': 640, 'height': 360, 'fps': 25.0} algorithm='yolov3'
